In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7e362df0d2e373943e126ded1a3f9ef34788d2efcf4e4f6df3b0b6718e8ef33d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("tpspark ").getOrCreate()

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


In [ ]:
rdd = sc.textFile("wash_dc_crime_incidents_2013.csv")

In [ ]:
rdd.count()

35898

In [ ]:
rdd.take(5)

['CCN,REPORTDATETIME,SHIFT,OFFENSE,METHOD,LASTMODIFIEDDATE,BLOCKSITEADDRESS,BLOCKXCOORD,BLOCKYCOORD,WARD,ANC,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,START_DATE,END_DATE',
 '04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM',
 '05047867,6/5/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,KNIFE,8/7/2015 8:32:22 AM,6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW,397769,144596,4,4B,FOURTH,402,17,,001901 4,001901,Precinct 59,4/15/2005 12:30:00 PM,',
 '07083463,7/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:32:15 AM,1800 - 1810 BLOCK OF COLUMBIA ROAD NW,396275,139402,1,1C,THIRD,303,1,ADAMS MORGAN,004002 1,004002,Precinct 25,7/14/2007 3:00:00 PM,',
 '09172197,4/8/2013 12:00:00 AM,MIDNIGHT,SEX ABUSE,OTHERS,8/7/2015 8:33:35 AM,2322 - 2499 BLOCK OF ONTARIO ROAD NW,396518,139335,1,

In [ ]:
rdd.first()

'CCN,REPORTDATETIME,SHIFT,OFFENSE,METHOD,LASTMODIFIEDDATE,BLOCKSITEADDRESS,BLOCKXCOORD,BLOCKYCOORD,WARD,ANC,DISTRICT,PSA,NEIGHBORHOODCLUSTER,BUSINESSIMPROVEMENTDISTRICT,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,START_DATE,END_DATE'

In [ ]:
header = rdd.first()
noHeaderRdd = rdd.filter(lambda row: row != header)
noHeaderRdd.first()

'04104147,4/16/2013 12:00:00 AM,MIDNIGHT,HOMICIDE,KNIFE,8/7/2015 8:34:01 AM,1500 - 1599 BLOCK OF 1ST STREET SW,398943,133729,6,6D,FIRST,105,9,,006400 2,006400,Precinct 127,7/27/2004 8:30:00 PM,7/27/2004 8:30:00 PM'

In [ ]:
noHeaderRdd.saveAsTextFile('wash_dc_crime_incidents_2013_without_header.csv')

In [ ]:
CrimeData = noHeaderRdd.map(lambda x : x.split(','))

In [ ]:
CrimeData.take(2)

[['04104147',
  '4/16/2013 12:00:00 AM',
  'MIDNIGHT',
  'HOMICIDE',
  'KNIFE',
  '8/7/2015 8:34:01 AM',
  '1500 - 1599 BLOCK OF 1ST STREET SW',
  '398943',
  '133729',
  '6',
  '6D',
  'FIRST',
  '105',
  '9',
  '',
  '006400 2',
  '006400',
  'Precinct 127',
  '7/27/2004 8:30:00 PM',
  '7/27/2004 8:30:00 PM'],
 ['05047867',
  '6/5/2013 12:00:00 AM',
  'MIDNIGHT',
  'SEX ABUSE',
  'KNIFE',
  '8/7/2015 8:32:22 AM',
  '6500  - 6599 BLOCK OF PINEY BRANCH ROAD NW',
  '397769',
  '144596',
  '4',
  '4B',
  'FOURTH',
  '402',
  '17',
  '',
  '001901 4',
  '001901',
  'Precinct 59',
  '4/15/2005 12:30:00 PM',
  '']]

In [ ]:
 from collections import namedtuple

CrimeDataTuple = namedtuple('CrimeData', ['date_string', 'time_string', 'offense', 'latitude', 'longitude'])

def map_line(line):
  cols = line.split(",")
  return CrimeDataTuple(date_string=cols[10], time_string=cols[11], offense=cols[3], latitude=cols[7], longitude=cols[8])

CrimeDataT = noHeaderRdd.map(map_line)

CrimeDataT.take(10)

[CrimeData(date_string='6D', time_string='FIRST', offense='HOMICIDE', latitude='398943', longitude='133729'),
 CrimeData(date_string='4B', time_string='FOURTH', offense='SEX ABUSE', latitude='397769', longitude='144596'),
 CrimeData(date_string='1C', time_string='THIRD', offense='SEX ABUSE', latitude='396275', longitude='139402'),
 CrimeData(date_string='1C', time_string='THIRD', offense='SEX ABUSE', latitude='396518', longitude='139335'),
 CrimeData(date_string='2A', time_string='SECOND', offense='SEX ABUSE', latitude='395232', longitude='136881'),
 CrimeData(date_string='5C', time_string='FIFTH', offense='SEX ABUSE', latitude='402158.31', longitude='138824.53'),
 CrimeData(date_string='7B', time_string='SIXTH', offense='SEX ABUSE', latitude='402837', longitude='133810'),
 CrimeData(date_string='8D', time_string='SEVENTH', offense='SEX ABUSE', latitude='398794', longitude='127300'),
 CrimeData(date_string='5E', time_string='FIFTH', offense='HOMICIDE', latitude='400331', longitude='140

In [ ]:
group_by_offense_rdd_tuple = CrimeDataT.groupBy(lambda x: x.offense)

In [ ]:
group_by_offense_rdd_tuple.take(5)

[('HOMICIDE', <pyspark.resultiterable.ResultIterable at 0x7e728a2cd2a0>),
 ('BURGLARY', <pyspark.resultiterable.ResultIterable at 0x7e72b07c0b20>),
 ('ROBBERY', <pyspark.resultiterable.ResultIterable at 0x7e728a28bc70>),
 ('THEFT F/AUTO', <pyspark.resultiterable.ResultIterable at 0x7e72b07c0f10>),
 ('ASSAULT W/DANGEROUS WEAPON',
  <pyspark.resultiterable.ResultIterable at 0x7e72b07c0a60>)]

In [ ]:
offense_counts = group_by_offense_rdd_tuple.map(lambda g: (g[0], len(g[1])))
offense_counts.take(5)

[('HOMICIDE', 104),
 ('BURGLARY', 3370),
 ('ROBBERY', 4071),
 ('THEFT F/AUTO', 10130),
 ('ASSAULT W/DANGEROUS WEAPON', 2309)]